In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np

data = np.genfromtxt('zcash-sprout-data-805518.csv', delimiter=',', skip_header=1)

In [ ]:
COIN = 100000000

def sum_chunk(x, chunk_size, axis=-1):
    shape = x.shape
    if axis < 0:
        axis += x.ndim
    shape = shape[:axis] + (-1, chunk_size) + shape[axis+1:]
    x = x.reshape(shape)
    return x.sum(axis=axis+1)

from_height = 0
to_height = 805000
chunk = 1000

# Slice the heights
height = sum_chunk(data[from_height:to_height,0], chunk)

num_types = sum_chunk(data[from_height:to_height,1:4].transpose(), chunk)
percent_types = num_types /  num_types.sum(axis=0).astype(float) * 100

value_shielded = data[from_height:to_height,7]
value_unshielded = data[from_height:to_height,10]
print(value_shielded.max() / COIN)
print(value_unshielded.max() / COIN)

value_shielded = sum_chunk(value_shielded, chunk)
value_unshielded = sum_chunk(value_unshielded, chunk)

# Rescale for chunk size
height = height / chunk
num_types = num_types / chunk
value_shielded = value_shielded / (chunk * COIN)
value_unshielded = value_unshielded / (chunk * COIN)

# Select the types char
chart_types = num_types.transpose()

sprout = chart_types[:,2]
sapling = chart_types[:,1]
transparent = chart_types[:,0]

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1, 1, 1)

ax.stackplot(height, sprout, sapling, transparent, labels=['Involves Sprout', 'Sapling + Transparent', 'Transparent-only'])
ax.set_title('Transaction types mined in the Zcash main chain')
ax.set_xlabel('Height')
ax.set_ylabel('Transactions per block (%d-block average)' % chunk)
ax.legend(loc='upper right')
ax.grid()
# ax.set_yscale('log')
plt.xticks(rotation=45)
plt.margins(x=0)
plt.savefig('tx-types-%d-%d-%d.png' % (from_height, to_height, chunk))

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1, 1, 1)

ax.fill_between(height, value_shielded, label='Shielding')
ax.fill_between(height, value_unshielded * -1, label='Unshielding')
ax.set_title('ZEC moving into and out of the Sprout value pool')
ax.set_xlabel('Height')
ax.set_ylabel('ZEC amount (%d-block average)' % chunk)
# ax.set_ylim(-1, 1)
ax.legend(loc='lower right')
ax.grid()
plt.xticks(rotation=45)
plt.margins(x=0)
plt.savefig('tx-values-%d-%d-%d.png' % (from_height, to_height, chunk))